Log into hugging_face to save model later

In [1]:
from huggingface_hub import login
login()

In [2]:
#@title Colab Extra Install { display-mode: "form" }
#%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [3]:
#@title Colab Extra Install { display-mode: "form" }
##%%capture
!pip install python-chess
!apt-get install stockfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 40.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=f9428d9c8bf6421d9120d47ad507dbd9feb997240a0e9e1e30b1a07c76c852d0
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  polyglot xboard | scid
The following NEW packages will be installed:
  stockfish
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 24.8 MB of archives.
After this operation, 47.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 stockfish amd64 14.1-1 [24.8 MB]
Fetched 24.8 MB in 2s (11.2 MB/s)
Selecting previously unselected package stockfish.
(Reading database ... 121713 files and directories cur

In [4]:
#@title Colab Extra Install { display-mode: "form" }
#%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 52.8 MB/s eta 0:00:00
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 121ms
Prepared 1 package in 491ms
Uninstalled 1 package in 264ms
Installed 1 package in 42ms
 - transformers==4.57.2
 + transformers==4.56.2
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 3ms
Prepared 1 package in 55ms
Uninstalled 1 package in 1ms
Installed 1 package in 5ms
 - trl==0.24.0
 + trl==0.22.2


In [5]:
#@title Load model config{ display-mode: "form" }
import os
from dotenv import load_dotenv
load_dotenv()

import yaml

# Path to your YAML config file
path = '/content/config.yaml'

def load_config(path: str):
    with open(path, 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config(path)
config_name = "qwen4b"
print("Selected config_name:", config_name)

match config_name:
    case "llama":
        config = config["llama_config"]
    case "phi":
        config = config["PHI_config"]
    case "mistral":
        config = config["mistral_config"]
    case "qwen7b":
        config = config["qwen7b_config"]
    case "qwen4b":
        config = config["qwen4b_config"]
    case _:
        raise ValueError("Check model name – perhaps the keyboard got excited.")

# Stockfish path from env
# stockfish_path = os.getenv("STOCKFISH_PATH")
stockfish_path= '/usr/games/stockfish'

print("STOCKFISH_PATH:", stockfish_path)

Selected config_name: qwen4b
STOCKFISH_PATH: /usr/games/stockfish


In [6]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset,concatenate_datasets
dataset = load_dataset("czovekboti/sft_chess")
print(len(dataset["train"]))

json_dataset = load_dataset("json", data_files="/content/training.json")

# Concatenate the 'train' splits from both datasets
dataset["train"] = concatenate_datasets([
    dataset["train"],
    json_dataset["train"]
])
print(len(dataset["train"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


chess_training_data_final.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

100


Generating train split: 0 examples [00:00, ? examples/s]

1661


In [7]:
print(len(dataset["train"]))

1661


In [8]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.8, # Reduce if out of memory
)





🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 12-02 20:07:17 [__init__.py:244] Automatically detected platform cuda.
ERROR 12-02 20:07:22 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Qwen3 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [9]:
# #add paddings
if tokenizer.pad_token is None:
   tokenizer.pad_token = tokenizer.eos_token or "<|pad|>"

model.resize_token_embeddings(len(tokenizer))

target_modules = [
    "q_proj","k_proj","v_proj","o_proj",
    "gate_proj","up_proj","down_proj",

]


model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = target_modules,
    lora_alpha = lora_rank *2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)


Unsloth 2025.11.6 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [10]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
# original_dataset = load_dataset("czovekboti/sft_chess", split="train")

split = dataset['train'].train_test_split(test_size=0.1, seed=42)
train_ds = split["train"]
test_ds = split["test"]
print(train_ds)
print(test_ds)

Dataset({
    features: ['fen', 'top_5_moves', 'answer'],
    num_rows: 1494
})
Dataset({
    features: ['fen', 'top_5_moves', 'answer'],
    num_rows: 167
})


In [11]:
start_tag = "<think>"
end_tag = "</think>"

In [12]:

SYSTEM_PROMPT = f"""You are a chess coach assistant. You will be given a board position in FEN format. Your job is to analyze the board and choose the best move from the candidate list.

Please follow this exact format in your response:

{start_tag}
(Brief explanation of what you see on the board — piece activity, threats, and candidate moves)
{end_tag}
<answer>
(best move written in correct SAN format, such as Nf3 or exd5)
</answer>

The move must be chosen from the list.
Do not use UCI format like e2e4 — only SAN notation like e4, Nf3, or O-O.
### Example:
FEN: rnbqkbnr/pppppppp/8/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 1
Best 5 moves are the following: d5, e5, Nf6, Nc6, c5
{start_tag}
White has just played e4 and developed the knight to f3. It’s Black’s turn. The e4 pawn is undefended. Capturing it with the pawn from d7 to d5 is a natural central counter.
{end_tag}
<answer>
d5
</answer>

Now solve the following position:
"""

def fen_color(fen: str) -> str:
    try:
        parts = fen.strip().split()
        side = parts[1]
        return "White" if side == "w" else "Black" if side == "b" else "Unknown"
    except Exception:
        return "Unknown"

# ---------- add this block after loading model/tokenizer ----------
import torch, re, pandas as pd
from datasets import load_dataset

# (optional but recommended) ensure a chat template is set for base models
try:
    from unsloth.chat_templates import get_chat_template
    if getattr(tokenizer, "chat_template", None) in (None, ""):
        tokenizer = get_chat_template(tokenizer, chat_template="qwen3")
except Exception:
    # fallback: minimal qwen-like template
    if getattr(tokenizer, "chat_template", None) in (None, ""):
        tokenizer.chat_template = (
            "{% for m in messages %}"
            "{% if m['role'] == 'system' %}<|im_start|>system\n{{ m['content'] }}<|im_end|>\n"
            "{% elif m['role'] == 'user' %}<|im_start|>user\n{{ m['content'] }}<|im_end|>\n"
            "{% elif m['role'] == 'assistant' %}<|im_start|>assistant\n{{ m['content'] }}<|im_end|>\n"
            "{% endif %}{% endfor %}{% if add_generation_prompt %}<|im_start|>assistant\n{% endif %}"
        )



# if tokenizer.pad_token_id is None:
#     tokenizer.pad_token = tokenizer.eos_token



def extract_move(text: str):
    m = ANSWER_RE.search(text or "")
    return m.group(1).strip() if m else None

def to_top5_set(v):
    if v is None:
        return set()
    if isinstance(v, (list, tuple, set)):
        return set(str(x).strip() for x in v if str(x).strip())
    parts = [p.strip() for p in str(v).replace(";", ",").split(",")]
    return set(p for p in parts if p)

def build_prompt(fen: str, color: str, top_5_moves):
    msgs = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"FEN: {fen}\nYou are with the following pieces: {color}\nBest 5 moves are the following: {', '.join(top_5_moves)}"},
    ]
    answer =  tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    print(answer)
    return answer

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
def generate_answer(the_model, prompt: str, max_new_tokens=128, temperature=0.7, top_p=0.95, top_k=40):
    the_model.eval()
    with torch.no_grad():
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        out = the_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=temperature > 0,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True,
        )
    gen_ids = out[0][inputs["input_ids"].shape[-1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=False)


print("=" * 80)
print("TESTING MODEL BEFORE TRAINING + SCORING")
print("=" * 80)

ANSWER_RE = re.compile(r"<answer>\s*(.*?)\s*</answer>", re.DOTALL | re.IGNORECASE)
REASONING_RE = re.compile(r"<think>\s*(.*?)\s*</think>", re.DOTALL | re.IGNORECASE)

def test_model(train_ds, model,answer_re,reasoning_re, max = len(train_ds)):
  i=0
  rows = []
  for example in train_ds:
        if i== max:
          return rows
        fen = example["fen"]
        color = fen_color(fen)
        top_5_moves = example.get("top_5_moves", [])  # ADD THIS LINE
        prompt = build_prompt(fen, color, top_5_moves)
        output = generate_answer(model, prompt, max_new_tokens=128, temperature=0.0, top_p=1.0, top_k=0)  # deterministic

        move = extract_move(output)
        has_reasoning = bool(REASONING_RE.search(output))
        has_answer_tag = bool(ANSWER_RE.search(output))

        top5 = to_top5_set(example.get("top_5_moves"))
        in_top5 = (move in top5) if move else False

        best_move = (example.get("best_move") or "").strip() or None
        equals_best = (move == best_move) if (move and best_move) else False

        score = 0
        score += 1 if has_reasoning else 0
        score += 1 if has_answer_tag else 0
        score += 1 if move else 0
        score += 2 if in_top5 else 0
        score += 2 if equals_best else 0

        print(f"\n{'='*80}")
        print(f"Example {i+1}/{len(train_ds)}")
        print(f"{'='*80}")
        print(f"FEN: {fen}")
        print(f"Color: {color}")
        print(f"\nGenerated Answer:\n{output}")

        rows.append({
            "idx": i,
            "FEN": fen,
            "Color": color,
            "Move": move or "",
            "HasReasoningTag": has_reasoning,
            "HasAnswerTag": has_answer_tag,
            "InTop5": in_top5,
            "EqualsBest": equals_best,
            "Score": score,
            "Top5Moves": ", ".join(sorted(top5)) if top5 else "",
            "BestMove": best_move or "",
            "RawOutput": output,
        })
        i+=1
  return rows




TESTING MODEL BEFORE TRAINING + SCORING


In [13]:
test_test = False
if test_test:
    rows = test_model(test_ds, model,ANSWER_RE,REASONING_RE,10)
    df = pd.DataFrame(rows).sort_values("Score", ascending=False).reset_index(drop=True)
    print("\n=== SCORE TABLE (top 10 rows) ===")
    print(df[["idx","Move","InTop5","EqualsBest","HasReasoningTag","HasAnswerTag","Score"]].head(50))

    # save
    df.to_csv("sft_answer_scoring_before_training.csv", index=False)
    print("\nSaved: sft_answer_scoring_before_training.csv")
    print("\nStarting training...\n")

In [14]:


# Toggle W&B logging (set to False if you don't want to log)
USE_WANDB = False

if USE_WANDB:
    import wandb
    wandb.login(key="0b2686da6f6fb1e01ee91ebffe6a73c459884b4e")
    os.environ["WANDB_LOG_MODEL"] = "end"
    os.environ["WANDB_PROJECT"] = "Chess_RL_Project"
    os.environ["WANDB_ENTITY"] = "czovekboti-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem"
    wandb.login()
    wandb.init(
        project="Chess_RL_Project",
        entity="",
        tags = "new sft",
        name="New sft test run",
        config={
            "model": config["model"],
            "max_seq_length": config['max_seq_length'],
            "lora_rank": lora_rank,
            "learning_rate": config["learning_rate"],
            "max_steps": config["max_steps"],
            "tag": "SFT"
        }
    )
    print("W&B initialized.")
else:
    print("W&B disabled.")

W&B disabled.


In [15]:
from datasets import Dataset
print(train_ds)
def format_dataset(ds):
    formatted_texts = []
    for ex in ds:
        fen = (ex.get("fen") or "").strip()
        ans = (ex.get("answer") or "").strip()
        ans = ans.replace("<reasoning>", "<think>").replace("</reasoning>", "</think>")
        top_5_moves = ex['top_5_moves']
        moves_string = ', '.join(top_5_moves)
        # print(moves_string)
        if not fen or not ans:
            continue
        color = fen_color(fen)
        messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"\nFEN: {fen}\nYou are with the following pieces: {color}\nBest 5 moves are the following: {', '.join(top_5_moves)}"},
                {"role": "assistant", "content": ans},

        ]
        formatted_texts.append({"conversations": messages})
    dataset = Dataset.from_list(formatted_texts)
    dataset = dataset.map(formatting_prompts_func, batched = True,)
    return dataset
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
final_train_ds = format_dataset(train_ds)
print(final_train_ds[0])
final_test_ds = format_dataset(test_ds)
print(final_test_ds[0])
print(len(final_train_ds))
print(len(final_test_ds))


Dataset({
    features: ['fen', 'top_5_moves', 'answer'],
    num_rows: 1494
})


Map:   0%|          | 0/1494 [00:00<?, ? examples/s]

{'conversations': [{'content': 'You are a chess coach assistant. You will be given a board position in FEN format. Your job is to analyze the board and choose the best move from the candidate list.\n\nPlease follow this exact format in your response:\n\n<think>\n(Brief explanation of what you see on the board — piece activity, threats, and candidate moves)\n</think>\n<answer>\n(best move written in correct SAN format, such as Nf3 or exd5)\n</answer>\n\nThe move must be chosen from the list.\nDo not use UCI format like e2e4 — only SAN notation like e4, Nf3, or O-O.\n### Example:\nFEN: rnbqkbnr/pppppppp/8/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 1\nBest 5 moves are the following: d5, e5, Nf6, Nc6, c5\n<think>\nWhite has just played e4 and developed the knight to f3. It’s Black’s turn. The e4 pawn is undefended. Capturing it with the pawn from d7 to d5 is a natural central counter.\n</think>\n<answer>\nd5\n</answer>\n\nNow solve the following position:\n', 'role': 'system'}, {'content': '\n

Map:   0%|          | 0/167 [00:00<?, ? examples/s]

{'conversations': [{'content': 'You are a chess coach assistant. You will be given a board position in FEN format. Your job is to analyze the board and choose the best move from the candidate list.\n\nPlease follow this exact format in your response:\n\n<think>\n(Brief explanation of what you see on the board — piece activity, threats, and candidate moves)\n</think>\n<answer>\n(best move written in correct SAN format, such as Nf3 or exd5)\n</answer>\n\nThe move must be chosen from the list.\nDo not use UCI format like e2e4 — only SAN notation like e4, Nf3, or O-O.\n### Example:\nFEN: rnbqkbnr/pppppppp/8/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 1\nBest 5 moves are the following: d5, e5, Nf6, Nc6, c5\n<think>\nWhite has just played e4 and developed the knight to f3. It’s Black’s turn. The e4 pawn is undefended. Capturing it with the pawn from d7 to d5 is a natural central counter.\n</think>\n<answer>\nd5\n</answer>\n\nNow solve the following position:\n', 'role': 'system'}, {'content': '\n

#setup:
- run 10 epoch
- eval,log every 100 steps
- save 300 steps


In [16]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_train_ds,
    eval_dataset=final_test_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        completion_only_loss=True,
        per_device_train_batch_size = 6,
        per_device_eval_batch_size = 6,
        gradient_accumulation_steps = 2,
        warmup_ratio = 0.05,
        num_train_epochs = 10, # run 10 epoch
        metric_for_best_model = "eval_loss",
        eval_strategy="steps",   # evaluate every N steps
        eval_steps=100,                 # <-- evaluate every 100 steps
        save_strategy="steps",        # <-- save every 300 steps
        save_steps= 300,
        save_total_limit = 3,
        learning_rate = 2e-4,
        logging_steps = 100,
        fp16 = True,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "./sft_outputs",
        report_to = "wandb", # Use TrackIO/WandB
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1494 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/167 [00:00<?, ? examples/s]

In [17]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,494 | Num Epochs = 10 | Total steps = 1,250
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 2 x 1) = 12
 "-____-"     Trainable parameters = 66,060,288 of 4,087,844,864 (1.62% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: czovekboti (czovekboti-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

Save model

In [ ]:
# Paste your "Write" token when prompted
repo_id = "czovekboti/stf_model"
model.push_to_hub(repo_id)

In [ ]:
trained_model = trainer.model

In [ ]:
for i in range(num_test_examples):
    example = original_dataset[i]
    fen = example["fen"]
    color = fen_color(fen)

    prompt = build_prompt(fen, color)
    output = generate_answer(trained_model, prompt, max_new_tokens=128, temperature=0.0, top_p=1.0, top_k=0)  # deterministic

    move = extract_move(output)
    has_reasoning = bool(REASONING_RE.search(output))
    has_answer_tag = bool(ANSWER_RE.search(output))

    top5 = to_top5_set(example.get("top_5_moves"))
    in_top5 = (move in top5) if move else False

    best_move = (example.get("best_move") or "").strip() or None
    equals_best = (move == best_move) if (move and best_move) else False
    score = 0
    score += 1 if has_reasoning else 0
    score += 1 if has_answer_tag else 0
    score += 1 if move else 0
    score += 2 if in_top5 else 0
    score += 2 if equals_best else 0

    print(f"\n{'='*80}")
    print(f"Example {i+1}/{num_test_examples}")
    print(f"{'='*80}")
    print(f"FEN: {fen}")
    print(f"Color: {color}")
    print(f"\nGenerated Answer:\n{output}")

    rows.append({
        "idx": i,
        "FEN": fen,
        "Color": color,
        "Move": move or "",
        "HasReasoningTag": has_reasoning,
        "HasAnswerTag": has_answer_tag,
        "InTop5": in_top5,
        "EqualsBest": equals_best,
        "Score": score,
        "Top5Moves": ", ".join(sorted(top5)) if top5 else "",
        "BestMove": best_move or "",
        "RawOutput": output,
    })
rows  = test_model(test_ds,trained_model,ANSWER_RE,REASONING_RE)
df = pd.DataFrame(rows).sort_values("Score", ascending=False).reset_index(drop=True)
print("\n=== SCORE TABLE (top 10 rows) ===")
print(df[["idx","Move","InTop5","EqualsBest","HasReasoningTag","HasAnswerTag","Score"]].head(50))
df.to_csv("sft_answer_scoring_after_training.csv", index=False)
